In [ ]:
import asyncio
import aiohttp
from asyncio_throttle import Throttler
import os
import dotenv


In [ ]:
%load_ext dotenv
%dotenv keys.env


api_key = os.getenv('bibsKey')


headers = {
    "Authorization": f"apikey {api_key}",
    "Accept": "application/json",
}


In [ ]:
mms_id = '991000022909705274'
holding_ids = ['2249469840005274', '2231971640005274']

In [ ]:
item_objects = []

In [ ]:
async def gather_tasks(urls, headers=None):
    """Gather tasks for making calls to the Alma API."""
    throttler = Throttler(rate_limit=20)
    #  Create a list of tasks
    tasks = []
    for url in urls:
        task = asyncio.create_task(make_calls(url, headers=headers, throttler=throttler))
        tasks.append(task)


    # Wait for all of the tasks to finish
    item_list = await asyncio.gather(*tasks)

    return item_list


In [ ]:
async def make_calls(url, headers=None, throttler=None):
    """Makes a GET call using the url passed in. The item object is parsed as JSON, 
    the inventory price is mapped to replacement cost, and the object is PUT back to Alma."""
    items = []
    async with throttler:
        try:
            async with aiohttp.ClientSession() as session:
                response = await session.get(url, headers=headers)

                # pull item object
                object = await response.json()
                items.append(object)
        except Exception as e:
            print(f'{e} for {url}')


    return items

In [ ]:
# Creates the URL endpoints needed to make citation requests.
def build_url(holding_ids, mms_id): 
    a = []
    for i in holding_ids:
        holding_id = i
        mms_id = mms_id

        url = f"https://api-na.hosted.exlibrisgroup.com/almaws/v1/bibs/{mms_id}/holdings/{holding_id}/items"

        a.append(url)

    return a

In [ ]:
for item in item_responses[0]:
    print(item['total_record_count'])

In [145]:
def count_items_by_base_status(json_payload):
  """Counts the number of items in a JSON payload that have a base_status of 1 or 0, and provides the process_type for each item.

  Args:
    json_payload: A JSON payload.

  Returns:
    A dictionary containing the following keys:
      * "holding_id": the holding record that contains the items.
      * "location": the permanent physical location associated with that holding id.
      * "items_in_place": A count of items with base status of 1 (in place).
      * "items_not_in_place": A count of items with a base_status of 0 (not in place).
      * "items_nip_process": A dictionary containing the process_type for each item with a base_status of 0.
  """

  # Initialize the counts and process_type lists.
  base_status_1_count = 0
  base_status_0_count = 0
  base_status_0_process_types = {}

  # Iterate over the items in the JSON payload.
  for i in json_payload:
    x = 0
    while x < i['total_record_count']:
      for item in i['item']:

        holding_id = item['holding_data']['holding_id']
        location = item['item_data']['location']['desc']
        base_status = item['item_data']['base_status']['value']
        process_type = item['item_data']['process_type']['value']        

        if base_status == '0':
            if process_type not in base_status_0_process_types:
              base_status_0_process_types[process_type] = 1
              x += 1
              base_status_0_count += 1
            else:
              process_count = base_status_0_process_types.get(process_type, 0)
              process_count += 1
              base_status_0_process_types[process_type] = process_count
              x += 1
              base_status_0_count += 1

        elif base_status  == '1':
            base_status_1_count += 1
            x += 1

  # Return a dictionary containing the base_status_0_count and base_status_0_process_types variables.
  return {
     'holding_id': holding_id,
     'location' : location,
    'items_in_place': base_status_1_count,
    'items_not_in_place': base_status_0_count,
    'items_nip_process': base_status_0_process_types
  }

In [ ]:
urls = build_url(holding_ids, mms_id) 

In [ ]:
item_responses = await gather_tasks(urls, headers=headers)

In [146]:
for response in item_responses:
   a =  count_items_by_base_status(response)

   print(f'Inventory counts for Holding ID: {a["holding_id"]} in {a["location"]}') 
   print(f'Items in place: {a["items_in_place"]}')
   print(f'Items not in place: {a["items_not_in_place"]}')
   print(f'For the items not in place, the process types of the remaining items are: {a["items_nip_process"]}') 
   print('\n')

Inventory counts for Holding ID: 2249469840005274 in 7 Day Tech
Items in place: 136
Items not in place: 544
For the items not in place, the process types of the remaining items are: {'MISSING': 68, 'LOAN': 340, 'LOST_LOAN': 136}


Inventory counts for Holding ID: 2231971640005274 in OCN Lab Desk
Items in place: 30
Items not in place: 0
For the items not in place, the process types of the remaining items are: {}


